In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import pdfplumber
import os


In [2]:
headers = {
    "User-Agent": "MI-Campaign-Finance-Research/1.0 (contact: jwh1097@gmail.com)"
}

base_url = "https://campaignfinance.us/ottawa.mi/"

id_dict = {}
link_dict = {}

committee_id = 0
internal_id = 0

search_url = f"https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID={committee_id}&cmdSubmit=Search"
documents_url = f"https://campaignfinance.us/ottawa.mi/iDocuments.php?iCommitteeID={internal_id}&cmdOk=View+Campaign+Statements"


In [3]:
df = pd.read_csv("Post-General_2024_CF_Submissions.csv")
candidate_ids = df.loc[df['Committee Type'] == 0]
candidate_ids.to_csv('candidate_committees.csv')


In [ ]:
committee_ids = candidate_ids["Committee ID"]
committee_ids

0     97221
1     96945
2     96961
3     96976
4         0
      ...  
95        0
96    97043
97    97030
98    97111
99    97030
Name: Committee ID, Length: 81, dtype: int64

## Get the internal Campain Finance committee \#

In [5]:
output = pd.read_csv("output.csv")
scraped_committees = output["committee_id"]
scraped_committees = list(scraped_committees.unique())
scraped_committees

[np.int64(96945),
 np.int64(96961),
 np.int64(96976),
 np.int64(97019),
 np.int64(97073),
 np.int64(96566),
 np.int64(97037),
 np.int64(96902),
 np.int64(96314),
 np.int64(97031),
 np.int64(97036),
 np.int64(97062),
 np.int64(96898),
 np.int64(97027),
 np.int64(97030),
 np.int64(96897),
 np.int64(96891),
 np.int64(97088),
 np.int64(97145),
 np.int64(97126),
 np.int64(97035),
 np.int64(96895),
 np.int64(97084),
 np.int64(97032),
 np.int64(97041),
 np.int64(97209),
 np.int64(97111),
 np.int64(96499),
 np.int64(97042),
 np.int64(97026),
 np.int64(96334),
 np.int64(97083),
 np.int64(97020),
 np.int64(97095),
 np.int64(97117),
 np.int64(97057),
 np.int64(97119),
 np.int64(96458),
 np.int64(97043),
 np.int64(97058),
 np.int64(96680),
 np.int64(97033),
 np.int64(97243),
 np.int64(97078),
 np.int64(96900),
 np.int64(96942),
 np.int64(96838),
 np.int64(97112),
 np.int64(96896),
 np.int64(97101),
 np.int64(97245),
 np.int64(97232),
 np.int64(97133),
 np.int64(97070),
 np.int64(97240),
 np.int64(

In [6]:
for id in committee_ids:
    
    committee_id = id

    if committee_id in scraped_committees:
        print("Skipped Committee ID:", committee_id)
        continue
    else:
        print(committee_id, "not scraped, fetching data")

    search_url = f"https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID={committee_id}&cmdSubmit=Search"
    committee_portal_pattern = re.compile(r'iCommitteePortal\.php\?iCommitteeID=(\d+)$')

    r = requests.get(search_url, headers=headers) # comment this line out while making edits to avoid sending too many requests to CF US
    soup = BeautifulSoup(r.text, 'html.parser')

    # get a link to each committee and save internal id
    for link in soup.find_all('a'):
        x =  link.get('href')
        match = committee_portal_pattern.search(x)
        if match:
            link_dict[committee_id] = base_url + x
            id_dict[committee_id] = int(match.group(1))

    # A brief pause out of respect
    time.sleep(random.uniform(1.0, 2.0))

97221 not scraped, fetching data
Skipped Committee ID: 96945
Skipped Committee ID: 96961
Skipped Committee ID: 96976
Skipped Committee ID: 0
Skipped Committee ID: 97019
Skipped Committee ID: 97073
Skipped Committee ID: 96566
Skipped Committee ID: 97037
Skipped Committee ID: 96902
Skipped Committee ID: 96314
Skipped Committee ID: 97031
Skipped Committee ID: 97036
Skipped Committee ID: 97062
Skipped Committee ID: 96898
Skipped Committee ID: 97027
Skipped Committee ID: 97030
Skipped Committee ID: 96897
Skipped Committee ID: 96891
Skipped Committee ID: 97088
Skipped Committee ID: 97145
Skipped Committee ID: 97126
Skipped Committee ID: 97035
Skipped Committee ID: 96895
Skipped Committee ID: 97084
Skipped Committee ID: 97032
Skipped Committee ID: 97041
Skipped Committee ID: 97209
Skipped Committee ID: 97111
Skipped Committee ID: 96499
Skipped Committee ID: 97042
Skipped Committee ID: 97026
Skipped Committee ID: 96334
Skipped Committee ID: 97083
Skipped Committee ID: 97020
Skipped Committee I

In [7]:
link_dict

{97221: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=417'}

In [8]:
id_dict

{97221: 417}

## Get list of Documents for each committee

In [10]:
df = pd.DataFrame(columns= ["committee_id", "document", "received", "due", "pages", "pdf_url"])

In [12]:
for id in id_dict:
    internal_id = id_dict[id]
    documents_url = f"https://campaignfinance.us/ottawa.mi/iDocuments.php?iCommitteeID={internal_id}&cmdOk=View+Campaign+Statements"
    
    resp = requests.get(documents_url)
    time.sleep(random.uniform(1.0, 2.0))

    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find("table", class_="DataList")

    records = []

    for row in table.find_all("tr")[1:]:
        cells = row.find_all("td")
        if len(cells) < 5:
            continue

        link = cells[4].find("a")

        record = {
            "committee_id": id,
            "document": cells[0].get_text(strip=True),
            "received": cells[1].get_text(strip=True),
            "due": cells[2].get_text(strip=True),
            "pages": cells[3].get_text(strip=True),
            "pdf_url": "https://campaignfinance.us" + link["href"] if link else None
        }
        if record["pdf_url"] is None:
            continue

        records.append(record)
    
    # Append to CSV instead of concatenating in memory
    if records:
        records_df = pd.DataFrame(records)
        records_df.to_csv("output.csv", mode='a', header=False, index=False)

pd.set_option("display.max_colwidth", None)


In [13]:
df = pd.read_csv('output.csv')
df["pdf"] = "/" + df["pdf_url"].str.split("/").str[-1]
df['pages'] = df['pages'].astype(int)

In [14]:
targets = [
    "Post-Election Stmt - 11/05/2024 General Election", 
    #"Post-Election Stmt - 11/05/2024 General Election (Amended)"
]

# Filter the dataframe
post_2024_df = df[df['document'].isin(targets)].copy()

In [15]:
post_2024_df.to_csv("filtered_output.csv")

In [16]:
post_2024_df["pages"].describe()

count    75.000000
mean      5.040000
std       3.465818
min       2.000000
25%       3.000000
50%       4.000000
75%       6.000000
max      19.000000
Name: pages, dtype: float64

## Download PDFs

In [17]:
# Get list of all entries already in the directory
entries = os.listdir('cf_stmnt')

# Filter out directories to keep only files
files = [f for f in entries if os.path.isfile(os.path.join('cf_stmnt', f))]
print(files[0])

2266.PDF


In [18]:
for url, pdf in zip(post_2024_df["pdf_url"], post_2024_df["pdf"]):
    if pdf[1:] in files:
        continue
    output_path = "cf_stmnt" + pdf
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    time.sleep(random.uniform(1.5, 3.0))
    with open(output_path, "wb") as f:
        f.write(response.content)
    print("Downloaded", output_path)
    
    

## Get the total Contributions and Expenditures for each election cycle

In [19]:
def value_extraction(text):
    cont_match = re.search(r"(\d[\d,.]*)\s*\n.*TOTAL CONTRIBUTIONS", text)
    exp_match = re.search(r"(\d[\d,.]*)\s*\n.*TOTAL EXPENDITURES", text)
    ikcont_match = re.search(r"(\d[\d,.]*)\s*\n.*In-Kind Contributions", text)
    ikexp_match = re.search(r"(\d[\d,.]*)\s*\n.*In-Kind Expenditures", text)
    return cont_match.group(1), exp_match.group(1), ikcont_match.group(1), ikexp_match.group(1)


In [21]:
total = {}
for pdf_name in post_2024_df["pdf"]:
    file_path = "cf_stmnt" + pdf_name # i.e., cf_stmnt/6005.PDF

    with pdfplumber.open(file_path) as pdf:
        first_page = pdf.pages[1]
        text = first_page.extract_text()
    
    try:
        total[pdf_name] = value_extraction(text)
    except:
        print("error retrieving totals from:", pdf_name)
    


error retrieving totals from: /3507.PDF
error retrieving totals from: /3682.PDF
error retrieving totals from: /3504.PDF
error retrieving totals from: /3485.PDF
error retrieving totals from: /3624.PDF
error retrieving totals from: /3676.PDF
error retrieving totals from: /3679.PDF
error retrieving totals from: /3633.PDF
error retrieving totals from: /3548.PDF
error retrieving totals from: /3879.PDF
error retrieving totals from: /3499.PDF
error retrieving totals from: /3536.PDF
error retrieving totals from: /4108.pdf


In [22]:
total

{'/2427.PDF': ('550.00', '495.29', '0.00', '0.00'),
 '/2425.PDF': ('317.59', '2,559.39', '0.00', '0.00'),
 '/2423.PDF': ('0.00', '0.00', '0.00', '0.00'),
 '/2990.PDF': ('22,579.24', '19,712.83', '0.00', '1,250.00'),
 '/2992.PDF': ('10,876.00', '9,563.45', '1,965.73', '0.00'),
 '/2994.PDF': ('20,310.00', '20,310.00', '0.00', '0.00'),
 '/2996.PDF': ('790.00', '690.00', '2,743.45', '0.00'),
 '/2998.PDF': ('1,800.00', '1,767.40', '0.00', '0.00'),
 '/3000.PDF': ('0.00', '0.00', '0.00', '0.00'),
 '/3003.PDF': ('6,479.95', '6,479.95', '3,172.11', '0.00'),
 '/3006.PDF': ('22,690.60', '20,229.02', '695.38', '0.00'),
 '/3008.PDF': ('12,612.33', '10,398.64', '0.00', '0.00'),
 '/3010.PDF': ('0.00', '14.25', '22.00', '0.00'),
 '/3012.PDF': ('27,179.03', '15,951.56', '390.08', '0.00'),
 '/3014.PDF': ('28,321.05', '22,776.79', '558.49', '0.00'),
 '/3016.PDF': ('16,885.34', '16,386.37', '88.00', '0.00'),
 '/3018.PDF': ('250.00', '3,706.56', '22.00', '0.00'),
 '/3020.PDF': ('37,885.00', '36,683.51', '5

In [28]:
post_2024_df[['contributions', 'expenditures', 'in-kind_contributions', 'in-kind_expenditures']] = (
    post_2024_df['pdf']
    .map(total)
    .apply(pd.Series)
)

cols_to_fix = ['contributions', 'expenditures', 'in-kind_contributions', 'in-kind_expenditures']

# Replace commas globally in those columns and convert
post_2024_df[cols_to_fix] = post_2024_df[cols_to_fix].replace({',': ''}, regex=True).astype(float)


In [41]:
post_2024_df.head()

,Unnamed: 0,committee_id,document,received,due,pages,pdf_url,pdf,contributions,expenditures,in-kind_contributions,in-kind_expenditures
3,3,97221,Post-Election Stmt - 11/05/2024 General Election,12/06/24,12/06/24,4,https://campaignfinance.us/MI-OTTAWA/IndexedDo...,/2266.PDF,1047.79,1024.29,300.0,0.0
19,19,96945,Post-Election Stmt - 11/05/2024 General Election,12/06/24,12/06/24,5,https://campaignfinance.us/MI-OTTAWA/IndexedDo...,/2427.PDF,550.00,495.29,0.0,0.0
49,49,96961,Post-Election Stmt - 11/05/2024 General Election,12/04/24,12/05/24,4,https://campaignfinance.us/MI-OTTAWA/IndexedDo...,/2425.PDF,317.59,2559.39,0.0,0.0
69,69,96976,Post-Election Stmt - 11/05/2024 General Election,11/13/24,12/05/24,2,https://campaignfinance.us/MI-OTTAWA/IndexedDo...,/2423.PDF,0.00,0.00,0.0,0.0
86,86,97019,Post-Election Stmt - 11/05/2024 General Election,12/04/24,12/05/24,3,https://campaignfinance.us/MI-OTTAWA/IndexedDo...,/2990.PDF,22579.24,19712.83,0.0,1250.0


In [29]:
post_2024_df.to_csv("filtered_output.csv")

In [30]:
print(post_2024_df['contributions'].describe())
print("total:", post_2024_df['contributions'].sum())

count        62.000000
mean      12154.288871
std       23919.441317
min           0.000000
25%         610.000000
50%        2519.460000
75%       13702.080000
max      119920.760000
Name: contributions, dtype: float64
total: 753565.91


In [31]:
print(post_2024_df['in-kind_contributions'].describe())
print("total:", post_2024_df['in-kind_contributions'].sum())

count      62.000000
mean      869.791613
std      1732.464020
min         0.000000
25%         0.000000
50%        22.000000
75%       698.845000
max      9131.630000
Name: in-kind_contributions, dtype: float64
total: 53927.08


In [32]:
print(post_2024_df['expenditures'].describe())
print("total:", post_2024_df['expenditures'].sum())

count        62.000000
mean      11400.316290
std       20466.414425
min           0.000000
25%         940.972500
50%        2569.350000
75%       13583.295000
max      103992.430000
Name: expenditures, dtype: float64
total: 706819.61


In [33]:
print(post_2024_df['in-kind_expenditures'].describe())
print("total:", post_2024_df['in-kind_expenditures'].sum())

count      62.000000
mean       20.161290
std       158.750159
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      1250.000000
Name: in-kind_expenditures, dtype: float64
total: 1250.0
